In [ ]:
from src.requirements import *

In [ ]:
from src.requirements import *

class FeatureEncoder(nn.Module):
    def __init__(self, in_channels=1, hidden_dim=128):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(in_channels, hidden_dim, kernel_size=3, stride=5, padding=1),
            nn.BatchNorm1d(hidden_dim),
            nn.GELU(),
        
            nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, stride=4, padding=1),
            nn.BatchNorm1d(hidden_dim),
            nn.GELU(),
        
            nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, stride=4, padding=1),
            nn.BatchNorm1d(hidden_dim),
            nn.GELU(),
        
            nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, stride=4, padding=1),
            nn.BatchNorm1d(hidden_dim),
            nn.GELU()            
        )
        
    def forward(self, x):
        return self.encoder(x)

class ContextModule(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, batch_first=True)

    def forward(self, z):
        output, _ = self.gru(z)
        return output

class Predictor(nn.Module):
    def __init__(self, hidden_dim, proj_dim):
        super().__init__()
        self.project = nn.Sequential(
            nn.Linear(proj_dim, proj_dim),
            nn.LayerNorm(proj_dim),
            nn.ReLU(),
            nn.Linear(proj_dim, proj_dim)
        )

    def forward(self, x):
        return self.project(x)

class SSLModel(nn.Module):
    def __init__(self, feat_dim=128, proj_dim=128, m=0.999):
        super().__init__()

        # online networks
        self.encoder = FeatureEncoder()
        self.context = ContextModule(feat_dim, feat_dim)        
        self.projector = nn.Sequential(
            nn.Linear(feat_dim, proj_dim),
            nn.LayerNorm(proj_dim),
            nn.ReLU(),
            nn.Linear(proj_dim, proj_dim)
        )
        self.predictor = Predictor(proj_dim, proj_dim)

        # momentum networks (exponential moving average)
        self.momentum = m
        self.encoder_m = copy.deepcopy(self.encoder)
        self.context_m = copy.deepcopy(self.context)
        self.projector_m = copy.deepcopy(self.projector)

        self._init_momentum_encoder()

    def _init_momentum_encoder(self):
        for p in self.encoder_m.parameters():
            p.requires_grad = False
        for p in self.context_m.parameters():
            p.requires_grad = False
        for p in self.projector_m.parameters():
            p.requires_grad = False

    @torch.no_grad()
    def update_momentum(self):
        m = self.momentum
        for online, target in [(self.encoder, self.encoder_m),
                               (self.context, self.context_m),
                               (self.projector, self.projector_m)]:
            for p_o, p_t in zip(online.parameters(), target.parameters()):
                p_t.data = p_t.data * m + p_o.data * (1.0 - m)

    def extract_features(self, x):
        z = self.encoder(x)
        z = z.transpose(1, 2)
        return self.context(z)

    def forward(self, x, mask_prob=0.065, mask_length=10):
        B = x.size(0)
    
        # online branch
        z1 = self.encoder(x).transpose(1, 2)
        T = z1.size(1)
    
        mask1 = compute_mask_indices(
            B, T,
            mask_prob=mask_prob,
            mask_length=mask_length,
            device=z1.device
        )
    
        z1 = apply_mask(z1, mask1)
        c1 = self.context(z1)
        c1_seq = self.context(z1)
        c1 = masked_mean(c1, mask1)
    
        h1 = self.projector(c1)
        p1 = self.predictor(h1)
    
        # momentum branch
        with torch.no_grad():
            z2 = self.encoder_m(x).transpose(1, 2)
    
            mask2 = compute_mask_indices(
                B, T,
                mask_prob=mask_prob,
                mask_length=mask_length,
                device=z2.device
            )
    
            z2 = apply_mask(z2, mask2)
            c2 = self.context_m(z2)
            c2 = masked_mean(c2, mask2)
    
            h2 = self.projector_m(c2)
    
        # calc loss
        loss = byol_loss(p1, h2) + 0.05 * variance_loss(c1_seq)
        return loss


def compute_mask_indices(B, T, mask_prob=0.05, mask_length=10, device="cpu"):
    mask = torch.zeros((B, T), dtype=torch.bool, device=device)
    num_masked_steps = int(mask_prob * T)
    num_spans = max(1, num_masked_steps // mask_length)

    for b in range(B):
        possible_starts = torch.arange(T - mask_length, device=device)
        perm = torch.randperm(len(possible_starts), device=device)
        span_starts = possible_starts[perm[:num_spans]]

        for s in span_starts:
            mask[b, s: s + mask_length] = True

    return mask

def variance_loss(z, eps=1e-4):
    z = z.reshape(-1, z.size(-1))
    std = torch.sqrt(z.var(dim=0, unbiased=False) + eps)
    return torch.mean(F.relu(1.0 - std))


def apply_mask(z, mask):
    z = z.clone()
    z[mask] = 0.0
    return z

def masked_mean(c, mask):
    valid = (~mask).unsqueeze(-1).float()
    return (c * valid).sum(dim=1) / valid.sum(dim=1).clamp(min=1.0)


In [ ]:
class SSLModel(nn.Module):
    def __init__(self, feat_dim=128, proj_dim=128, m=0.999):
        super().__init__()

        self.encoder = FeatureEncoder(hidden_dim=feat_dim)
        self.context = ContextModule(feat_dim, feat_dim)
        
        self.projector = nn.Sequential(
            nn.Linear(feat_dim, proj_dim),
            nn.LayerNorm(proj_dim),
            nn.GELU(),
            nn.Linear(proj_dim, proj_dim)
        )
        self.predictor = Predictor(proj_dim, proj_dim)

        self.momentum = m
        self.encoder_m = copy.deepcopy(self.encoder)
        self.projector_m = copy.deepcopy(self.projector)
        
        self.mask_token = nn.Parameter(torch.zeros(1, 1, feat_dim))
        nn.init.normal_(self.mask_token, std=0.02)

        self._init_momentum_encoder()

    def _init_momentum_encoder(self):
        for p in list(self.encoder_m.parameters()) + list(self.projector_m.parameters()):
            p.requires_grad = False

    @torch.no_grad()
    def update_momentum(self):
        m = self.momentum
        for online, target in [(self.encoder, self.encoder_m),
                               (self.projector, self.projector_m)]:
            for p_o, p_t in zip(online.parameters(), target.parameters()):
                p_t.data = p_t.data * m + p_o.data * (1.0 - m)

    def forward(self, x, mask_prob=0.065, mask_length=10):
        B, C, L = x.size()
        
        z = self.encoder(x).transpose(1, 2)  # [B, T, feat_dim]
        T = z.size(1)

        mask = compute_mask_indices(B, T, mask_prob, mask_length, z.device)
        
        z_masked = z.clone()
        z_masked[mask] = self.mask_token.to(z.dtype)
        
        c = self.context(z_masked)
        p = self.predictor(self.projector(c)) # [B, T, proj_dim]

        with torch.no_grad():
            z_m = self.encoder_m(x).transpose(1, 2)
            target_h = self.projector_m(z_m) # [B, T, proj_dim]

        loss = self.byol_masked_loss(p, target_h, mask)
        
        loss += 0.05 * variance_loss(z)
        
        return loss

    def byol_masked_loss(self, p, z, mask):
        p_masked = p[mask] 
        z_masked = z[mask]
        
        p_norm = F.normalize(p_masked, dim=-1)
        z_norm = F.normalize(z_masked, dim=-1)
        
        return (2 - 2 * (p_norm * z_norm).sum(dim=-1)).mean()

In [1]:
from __future__ import division
from random import random
from math import pi
import matplotlib.pyplot as plt
import sys


def rain_drop(length_of_field=1):
    """
    Simulate a random rain drop within the square field.
    """
    return [
        (0.5 - random()) * length_of_field,
        (0.5 - random()) * length_of_field
    ]


def is_point_in_circle(point, length_of_field=1):
    """
    Check if a point lies within the inscribed circle.
    """
    return point[0] ** 2 + point[1] ** 2 <= (length_of_field / 2) ** 2


def plot_rain_drops(drops_in_circle, drops_out_of_circle, length_of_field=1, format="png"):
    """
    Plot the raindrops inside and outside the circle.
    """
    plt.figure()
    plt.xlim(-length_of_field / 2, length_of_field / 2)
    plt.ylim(-length_of_field / 2, length_of_field / 2)

    plt.scatter(
        [e[0] for e in drops_in_circle],
        [e[1] for e in drops_in_circle],
        color="blue",
        label="Drops in circle"
    )

    plt.scatter(
        [e[0] for e in drops_out_of_circle],
        [e[1] for e in drops_out_of_circle],
        color="black",
        label="Drops out of circle"
    )

    plt.legend(loc="center")
    total = len(drops_in_circle) + len(drops_out_of_circle)
    pi_est = 4 * len(drops_in_circle) / total

    plt.title(
        f"{total} drops: {len(drops_in_circle)} in circle, "
        f"π ≈ {pi_est:.4f}"
    )
    plt.savefig(f"rain_drops.{format}")
    plt.close()


def plot_pi_estimates(pi_estimate, total_drops):
    """
    Plot π estimates versus number of drops.
    """
    plt.figure()
    plt.scatter(range(1, total_drops + 1), pi_estimate, s=1)
    plt.hlines(pi, 0, total_drops, color="black", linestyle="dashed", label="Actual π")
    plt.xlim(0, total_drops)

    plt.title("π estimate vs number of rain drops")
    plt.xlabel("Number of rain drops")
    plt.ylabel("π")
    plt.legend()
    plt.savefig(f"pi_estimate_{total_drops}_drops.png")
    plt.close()


def rain(number_of_drops, length_of_field=1, plot=True, dynamic=False):
    """
    Simulate raindrops to estimate π.
    """
    number_of_drops_in_circle = 0
    drops_in_circle = []
    drops_out_of_circle = []
    pi_estimate = []

    for k in range(number_of_drops):
        drop = rain_drop(length_of_field)

        if is_point_in_circle(drop, length_of_field):
            drops_in_circle.append(drop)
            number_of_drops_in_circle += 1
        else:
            drops_out_of_circle.append(drop)

        pi_estimate.append(4 * number_of_drops_in_circle / (k + 1))

        if dynamic:
            plot_rain_drops(drops_in_circle, drops_out_of_circle, length_of_field)

    if plot and not dynamic:
        plot_rain_drops(drops_in_circle, drops_out_of_circle, length_of_field)
        plot_pi_estimates(pi_estimate, number_of_drops)

    return number_of_drops_in_circle, number_of_drops


if __name__ == "__main__":
    try:
        if len(sys.argv) > 1:
            number_of_drops = int(sys.argv[1])
        else:
            number_of_drops = int(input("Enter the number of drops: "))

        result = rain(number_of_drops, plot=True, dynamic=False)

        print("----------------------")
        print(f"{number_of_drops} drops")
        print(f"π estimated as: {4 * result[0] / result[1]:.6f}")
        print("----------------------")

    except ValueError:
        print("Invalid input! Please provide an integer.")

Invalid input! Please provide an integer.
